# VirES Python Client API

<a id="top"/>

## Contents
- [Access token configuration](#access_token_configuration)
- [Send requests to the server](#SwarmRequest)
    - [Get the available collections](#SwarmRequest.available_collections)
    - [Get the available measurements](#SwarmRequest.available_measurements)
    - [Get the available auxiliaries](#SwarmRequest.available_auxiliaries)
    - [Get the available magnetic models](#SwarmRequest.available_models)
    - [Get information about one or mode models](#SwarmRequest.get_model_info)
    - [Get the orbit number](#SwarmRequest.get_orbit_number)
    - [Get times for orbits](#SwarmRequest.get_times_for_orbits)
    - [Set collections](#SwarmRequest.set_collection)
    - [Set products](#SwarmRequest.set_products)
    - [Set/clear filters](#SwarmRequest.set_range_filter)
    - [Send request to the server](#SwarmRequest.get_between)
- [`ReturnedData`](#ReturnedData)
    - [`ReturnedData.sources`](#ReturnedData.sources)
    - [`ReturnedData.contents`](#ReturnedData.contents)
    - [`ReturnedData.filetype`](#ReturnedData.filetype)
    - [`ReturnedData.magnetic_models`](#ReturnedData.magnetic_models)
    - [`ReturnedData.range_filters`](#ReturnedData.range_filters)
    - [`ReturnedData.as_dataframe()`](#ReturnedData.as_dataframe)
    - [`ReturnedData.as_xarray()`](#ReturnedData.as_xarray)
    - [`ReturnedData.to_file()`](#ReturnedData.to_file)
    - [`ReturnedData.to_files()`](#ReturnedData.to_files)
- [`ReturnedDataFile`](#ReturnedDataFile)
    - [`ReturnedDataFile._file`](#ReturnedDataFile._file)
    - [`ReturnedDataFile.filetype`](#ReturnedDataFile.filetype)
    - [`ReturnedDataFile.as_dataframe()`](#ReturnedDataFile.as_dataframe)
    - [`ReturnedDataFile.as_xarray()`](#ReturnedDataFile.as_xarray)
    - [`ReturnedDataFile.to_file()`](#ReturnedDataFile.to_file)
- [`ClientConfig`](#ClientConfig)
    - [`ClientConfig.path`](#ClientConfig.path)
    - [`ClientConfig.default_url`](#ClientConfig.default_url)
    - [`ClientConfig.set_site_config()`](#ClientConfig.set_site_config)
    - [`ClientConfig.get_site_config()`](#ClientConfig.get_site_config)
    - [`ClientConfig.save()`](#ClientConfig.save)
- [`DataUpload`](#DataUpload)
    - [`DataUpload.post()`](#DataUpload.post)
    - [`DataUpload.ids`](#DataUpload.ids)
    - [`DataUpload.get()`](#DataUpload.get)
    - [`DataUpload.set_constant_parameters()`](#DataUpload.set_constant_parameters)
    - [`DataUpload.get_constant_parameters()`](#DataUpload.get_constant_parameters)
    - [`DataUpload.delete()`](#DataUpload.delete)
    - [`DataUpload.clear()`](#DataUpload.clear)

[TOP](#top)

<a id="access_token_configuration"/>

## Access token configuration

Before using the client, it is necessary to set the access token. This can be done using the *set_token* function. First, the function must be imported from the *viresclient* package:

In [ ]:
from viresclient import set_token

Let's use `?` to get help on the function: 

In [1]:
set_token?

Object `set_token` not found.


The function has the following arguments:
- **url** (*str*, optional): server URL (default value: `https://vires.services/ows`).
- **token** (*str*, optional): token string obtained from the VirES access token management page (https://vires.services/accounts/tokens/). If this parameter is not set, the user is prompted to enter its value interactively.
- **set_default** (*bool*, optional): if `True`, server identified by *url* is configured as default server (default `False`).

Since the *url* parameter is by default set to the correct value we don't need to specify it. We can avoid also to specify the *token* parameter because this value will be asked interactively. Moreover, setting `set_default=True` we can configure this URL as default in the configuration file and avoid to specify it while sending requests to the server.

In [ ]:
set_token(set_default=True)

Setting access token for https://vires.services/ows ...
Generate a token at https://vires.services/accounts/tokens/


[TOP](#top)

<a id="SwarmRequest"/>

## Send requests to the server

Requests are handled via the *SwarmRequest* object. This allows to create a request and to download data according to the input parameters. Before using it, it must be imported from the *viresclient* package.

In [16]:
from viresclient import SwarmRequest

Let's use `?` to get help on the object: 

In [2]:
SwarmRequest?

Init signature:
SwarmRequest(
    url=None,
    username=None,
    password=None,
    token=None,
    config=None,
    logging_level='NO_LOGGING',
)
Docstring:     
Handles the requests to and downloads from the server.

Example usage::

    from viresclient import SwarmRequest
    # Set up connection with server
    request = SwarmRequest()
    # Set collection to use
    request.set_collection("SW_OPER_MAGA_LR_1B")
    # Set mix of products to fetch:
    #  measurements (variables from the given collection)
    #  models (magnetic model predictions at spacecraft sampling points)
    #  auxiliaries (variables available with any collection)
    request.set_products(
        measurements=["F", "B_NEC"],
        models=["CHAOS-Core"],
        auxiliaries=["QDLat", "QDLon"],
        sampling_step="PT10S"
    )
    # Fetch data from a given time interval
    data = request.get_between(
        start_time="2014-01-01T00:00",
        end_time="2014-01-01T01:00"
    )
    # Load the data as a

*SwarmRequest* object has the following arguments:

- **url** (*str*, optional): if not provided, the request will be sent to the default URL set in the `~/.viresclient.ini` configuration file.
- **username** (*str*, optional): username. The usage of username and password is deprecated and will be removed in future releases.
- **password** (*str*, optional): password. The usage of username and password is deprecated and will be removed in future releases.
- **token** (*str*, optional): token string obtained from the VirES access token management page (https://vires.services/accounts/tokens/). If this parameter is not specified and it is not set in the configuration file, the user is prompted to enter its value interactively.
- **config** (*str* or *ClientConfig*, optional): viresclient configuration. By default, it is read from `~/.viresclient.ini`.
- **logging_level** (*str*, optional): set the logging level. Allowed values are: `DEBUG`, `INFO`, `WARNING`, `ERROR`, `NO_LOGGING` (default).

After configuring the access token using *set_token* function (see [Access token configuration](#access_token_configuration)) , the *SwarmRequest* object can be instantiated as follows:

In [7]:
request = SwarmRequest()

The software automatically retrieves the access token for the `https://vires.services/ows` default URL configured in the `~/.viresclient.ini` file:

```ini
[https://vires.services/ows]
token = <token>

[default]
url = https://vires.services/ows
```

[TOP](#top)

<a id="SwarmRequest.available_collections"/>

### Get the available collections

Swarm data are organized in *collections*. Each collection is related to a Swarm file type (e.g. collection *SW_OPER_MAGA_LR_1B* is related to file type *MAGA_LR_1B*). The list of the available collections are provided invoking the *available_collections* method.

In [25]:
# Import SwarmClient if not already done
from viresclient import SwarmRequest

# Get info on SwarmRequest.available_collections
SwarmRequest.available_collections?

Signature: SwarmRequest.available_collections(self, details=True)
Docstring:
Show details of available collections.

Args:
    details (bool): If True then print a nice output.
        If False then return a list of available collections.
File:      /opt/conda/lib/python3.7/site-packages/viresclient/_client_swarm.py
Type:      function


Arguments:

- **details** (*bool*, optional): if `True` (default), the method prints the list of all the available collections and the related details. If `False`, it returns the available collections as a *list*.

E.g. print the list of the available collections with the details:

In [19]:
# Create the request object
request = SwarmRequest()

# Get the available collections
request.available_collections()

General References:
 Swarm Data Handbook, https://earth.esa.int/web/guest/missions/esa-eo-missions/swarm/data-handbook 
 The Swarm Satellite Constellation Application and Research Facility (SCARF) and Swarm data products, https://doi.org/10.5047/eps.2013.07.001 
 Swarm Science Data Processing and Products (2013), https://link.springer.com/journal/40623/65/11/page/1 
 Special issue “Swarm science results after 2 years in space (2016), https://www.springeropen.com/collections/swsr 
 Earth's Magnetic Field: Understanding Geomagnetic Sources from the Earth's Interior and its Environment (2017), https://link.springer.com/journal/11214/206/1/page/1 

MAG
   SW_OPER_MAGA_LR_1B
   SW_OPER_MAGB_LR_1B
   SW_OPER_MAGC_LR_1B
 https://earth.esa.int/web/guest/missions/esa-eo-missions/swarm/data-handbook/level-1b-product-definitions#MAGX_LR_1B_Product 

EFI
   SW_OPER_EFIA_LP_1B
   SW_OPER_EFIB_LP_1B
   SW_OPER_EFIC_LP_1B
 https://earth.esa.int/web/guest/missions/esa-eo-missions/swarm/data-handbook/l

E.g. get the list of available collections without details:

In [20]:
request.available_collections(details=False)

['SW_OPER_MAGA_LR_1B',
 'SW_OPER_MAGB_LR_1B',
 'SW_OPER_MAGC_LR_1B',
 'SW_OPER_EFIA_LP_1B',
 'SW_OPER_EFIB_LP_1B',
 'SW_OPER_EFIC_LP_1B',
 'SW_OPER_IBIATMS_2F',
 'SW_OPER_IBIBTMS_2F',
 'SW_OPER_IBICTMS_2F',
 'SW_OPER_TECATMS_2F',
 'SW_OPER_TECBTMS_2F',
 'SW_OPER_TECCTMS_2F',
 'SW_OPER_FACATMS_2F',
 'SW_OPER_FACBTMS_2F',
 'SW_OPER_FACCTMS_2F',
 'SW_OPER_FAC_TMS_2F',
 'SW_OPER_EEFATMS_2F',
 'SW_OPER_EEFBTMS_2F',
 'SW_OPER_IPDAIRR_2F',
 'SW_OPER_IPDBIRR_2F',
 'SW_OPER_IPDCIRR_2F']

[TOP](#top)

<a id="SwarmRequest.available_measurements"/>

### Get the available measurements

It is possible to get the available measurements using the *available_measurements* method.

In [24]:
# Import SwarmClient if not already done
from viresclient import SwarmRequest

# Get info on SwarmRequest.available_measurements
SwarmRequest.available_measurements?

Signature: SwarmRequest.available_measurements(self, collection=None)
Docstring:
Returns a list of the available measurements for the chosen collection.

Args:
    collection (str): one of: ("MAG", "EFI", "IBI", "TEC", "FAC", "EEF")
File:      /opt/conda/lib/python3.7/site-packages/viresclient/_client_swarm.py
Type:      function


Arguments:

- **collection** (*str*, optional). If indicated, returns the available measurements for this collection as a *list*. It can be set to: `MAG`, `EFI`, `IBI`, `TEC`, `FAC`, `EEF`, `IPD` or one of the collections returned by the *available_collections* method. If not indicated, it returns the available measurements for all the collections as a *dict*.

E.g. get the list of measurements for the `MAG` collections:

In [26]:
# Create the request object
request = SwarmRequest()

# Get the available measurements
request.available_measurements('MAG')

['F',
 'dF_AOCS',
 'dF_other',
 'F_error',
 'B_VFM',
 'B_NEC',
 'dB_Sun',
 'dB_AOCS',
 'dB_other',
 'B_error',
 'q_NEC_CRF',
 'Att_error',
 'Flags_F',
 'Flags_B',
 'Flags_q',
 'Flags_Platform',
 'ASM_Freq_Dev']

Setting `collection=SW_OPER_MAGA_LR_1B` we obtain the same result:

In [7]:
request.available_measurements('SW_OPER_MAGA_LR_1B')

['F',
 'dF_AOCS',
 'dF_other',
 'F_error',
 'B_VFM',
 'B_NEC',
 'dB_Sun',
 'dB_AOCS',
 'dB_other',
 'B_error',
 'q_NEC_CRF',
 'Att_error',
 'Flags_F',
 'Flags_B',
 'Flags_q',
 'Flags_Platform',
 'ASM_Freq_Dev']

E.g. get all the available measurements:

In [8]:
request.available_measurements()

{'MAG': ['F',
  'dF_AOCS',
  'dF_other',
  'F_error',
  'B_VFM',
  'B_NEC',
  'dB_Sun',
  'dB_AOCS',
  'dB_other',
  'B_error',
  'q_NEC_CRF',
  'Att_error',
  'Flags_F',
  'Flags_B',
  'Flags_q',
  'Flags_Platform',
  'ASM_Freq_Dev'],
 'EFI': ['U_orbit',
  'Ne',
  'Ne_error',
  'Te',
  'Te_error',
  'Vs',
  'Vs_error',
  'Flags_LP',
  'Flags_Ne',
  'Flags_Te',
  'Flags_Vs'],
 'IBI': ['Bubble_Index',
  'Bubble_Probability',
  'Flags_Bubble',
  'Flags_F',
  'Flags_B',
  'Flags_q'],
 'TEC': ['GPS_Position',
  'LEO_Position',
  'PRN',
  'L1',
  'L2',
  'P1',
  'P2',
  'S1',
  'S2',
  'Elevation_Angle',
  'Absolute_VTEC',
  'Absolute_STEC',
  'Relative_STEC',
  'Relative_STEC_RMS',
  'DCB',
  'DCB_Error'],
 'FAC': ['IRC',
  'IRC_Error',
  'FAC',
  'FAC_Error',
  'Flags',
  'Flags_F',
  'Flags_B',
  'Flags_q'],
 'EEF': ['EEF', 'RelErr', 'flags'],
 'IPD': ['Ne',
  'Te',
  'Background_Ne',
  'Foreground_Ne',
  'PCP_flag',
  'Grad_Ne_at_100km',
  'Grad_Ne_at_50km',
  'Grad_Ne_at_20km',
  'Grad

[TOP](#top)

<a id="SwarmRequest.available_auxiliaries"/>

### Get the available auxiliaries

It is possible to get the available auxiliaries using the *available_auxiliaries* method.

In [27]:
# Import SwarmClient if not already done
from viresclient import SwarmRequest

# Get info on SwarmRequest.available_auxiliaries
SwarmRequest.available_auxiliaries?

Signature: SwarmRequest.available_auxiliaries(self)
Docstring:
Returns a list of the available auxiliary parameters.
        
File:      /opt/conda/lib/python3.7/site-packages/viresclient/_client_swarm.py
Type:      function


This method does not accept any argument and provides the available auxiliaries as a *list*:

In [28]:
# Create the request object
request = SwarmRequest()

# Get the list of available auxiliaries
request.available_auxiliaries()

['Timestamp',
 'Latitude',
 'Longitude',
 'Radius',
 'Spacecraft',
 'OrbitDirection',
 'QDOrbitDirection',
 'SyncStatus',
 'Kp10',
 'Kp',
 'Dst',
 'F107',
 'IMF_BY_GSM',
 'IMF_BZ_GSM',
 'IMF_V',
 'F10_INDEX',
 'OrbitSource',
 'OrbitNumber',
 'AscendingNodeTime',
 'AscendingNodeLongitude',
 'QDLat',
 'QDLon',
 'QDBasis',
 'MLT',
 'SunDeclination',
 'SunHourAngle',
 'SunRightAscension',
 'SunAzimuthAngle',
 'SunZenithAngle',
 'SunLongitude',
 'SunVector',
 'DipoleAxisVector',
 'NGPLatitude',
 'NGPLongitude',
 'DipoleTiltAngle',
 'UpwardCurrent',
 'TotalCurrent',
 'DivergenceFreeCurrentFunction']

[TOP](#top)

<a id="SwarmRequest.available_models"/>

### Get the available magnetic models

The available magnetic models can be obtained via the *available_models* method.

In [29]:
# Import SwarmClient if not already done
from viresclient import SwarmRequest

# Get info on SwarmRequest.available_models
SwarmRequest.available_models?

Signature:
SwarmRequest.available_models(
    self,
    param=None,
    details=True,
    nice_output=True,
)
Docstring:
Show details of avalable models.

If details is True, return a dictionary of model names and details.
If nice_output is True, the dictionary is printed nicely.
If details is False, return a list of model names.
If param is set, filter to only return entries including this

Note:
    |  F = Fast-Track Products
    |  C = Comprehensive Inversion
    |  D = Dedicated Chain
    |  MCO = Core / main
    |  MLI = Lithosphere
    |  MMA = Magnetosphere
    |  MIO = Ionosphere

Args:
    param (str): one of "F C D MCO MLI MMA MIO"
    details (bool): True for a dict of details, False for a brief list
    nice_output (bool): If True, just print the dict nicely
File:      /opt/conda/lib/python3.7/site-packages/viresclient/_client_swarm.py
Type:      function


The method accepts the following arguments:

- **param** (*str*, optional): it can be set to one of `F`, `C`, `D`, `MCO`, `MLI`, `MMA`, `MIO` to filter all the available magnetic models.
- **details** (*bool*, optional): if it is set to `True` (default), it gets the models and their details. If `False`, it returns only the models as a *list*.
- **nice_output** (*bool*, optional): if it is set to `True` (default), it prints the models and their details. If `False`, it returns the models as a dictionary (if `details=True`) or a s a list (if `details=False`).

E.g. get the list of all the available models and details

In [30]:
# Create the request object
request = SwarmRequest()

# Get available models
request.available_models()

CHAOS-6-Core = 'CHAOS-6-Core'(max_degree=20,min_degree=1)
  START: 1997-02-07T05:31:12.000003Z
  END:   2020-04-21T02:49:20.639999Z
DESCRIPTION:
CHAOS Core field
 deprecated model identifier, use CHAOS-Core instead
SOURCES:
  SW_OPER_MCO_SHA_2X_19970101T000000_20200419T235959_0701

CHAOS-6-MMA-Primary = 'CHAOS-6-MMA-Primary'(max_degree=2,min_degree=1)
  START: 2012-01-01T00:00:00Z
  END:   2020-02-01T23:30:00Z
DESCRIPTION:
CHAOS Primary (external) magnetospheric field
 deprecated model identifier, use CHAOS-MMA-Primary instead
SOURCES:
  SW_OPER_MMA_CHAOS__20120101T000000_20121231T233000_0608
  SW_OPER_MMA_CHAOS__20130101T000000_20131231T233000_0608
  SW_OPER_MMA_CHAOS__20140101T000000_20141231T233000_0608
  SW_OPER_MMA_CHAOS__20150101T000000_20151231T233000_0608
  SW_OPER_MMA_CHAOS__20160101T000000_20161231T233000_0608
  SW_OPER_MMA_CHAOS__20170101T000000_20171231T233000_0608
  SW_OPER_MMA_CHAOS__20180101T000000_20181231T233000_0701
  SW_OPER_MMA_CHAOS__20190101T000000_20200201T233000

E.g. print the list of the available `F` (i.e. Fast-Track) models and their details:

In [21]:
request.available_models('F')

MMA_SHA_2F-Primary = 'MMA_SHA_2F-Primary'(max_degree=1,min_degree=1)
  START: 2013-11-25T11:15:00Z
  END:   2020-01-20T21:45:00Z
DESCRIPTION:
[Fast-Track Product]: Primary (external) magnetospheric field
 Rapid modelling of the large-scale magnetospheric field from Swarm satellite data, https://doi.org/10.5047/eps.2013.09.003 
SOURCES:
  SW_OPER_MMA_SHA_2F_20131125T103000_20140101T000000_0108
  SW_OPER_MMA_SHA_2F_20140101T000000_20150101T000000_0108
  SW_OPER_MMA_SHA_2F_20150101T000000_20160101T000000_0108
  SW_OPER_MMA_SHA_2F_20160101T000000_20170101T000000_0108
  SW_OPER_MMA_SHA_2F_20170101T000000_20180101T000000_0108
  SW_OPER_MMA_SHA_2F_20180101T000000_20190101T000000_0108
  SW_OPER_MMA_SHA_2F_20190101T000000_20200101T000000_0108
  SW_OPER_MMA_SHA_2F_20200101T000000_20200120T223000_0108

MMA_SHA_2F-Secondary = 'MMA_SHA_2F-Secondary'(max_degree=1,min_degree=1)
  START: 2013-11-25T11:15:00Z
  END:   2020-01-20T21:45:00Z
DESCRIPTION:
[Fast-Track Product]: Secondary (internal/induced) 

E.g. get the If the list of models of type `F` as a dictionary:

In [22]:
request.available_models(param='F', nice_output=False)

{'MMA_SHA_2F-Primary': {'description': ('[Fast-Track Product]: Primary (external) magnetospheric field',
   ' Rapid modelling of the large-scale magnetospheric field from Swarm satellite data, https://doi.org/10.5047/eps.2013.09.003 '),
  'details': {'sources': ['SW_OPER_MMA_SHA_2F_20131125T103000_20140101T000000_0108',
    'SW_OPER_MMA_SHA_2F_20140101T000000_20150101T000000_0108',
    'SW_OPER_MMA_SHA_2F_20150101T000000_20160101T000000_0108',
    'SW_OPER_MMA_SHA_2F_20160101T000000_20170101T000000_0108',
    'SW_OPER_MMA_SHA_2F_20170101T000000_20180101T000000_0108',
    'SW_OPER_MMA_SHA_2F_20180101T000000_20190101T000000_0108',
    'SW_OPER_MMA_SHA_2F_20190101T000000_20200101T000000_0108',
    'SW_OPER_MMA_SHA_2F_20200101T000000_20200120T223000_0108'],
   'expression': "'MMA_SHA_2F-Primary'(max_degree=1,min_degree=1)",
   'validity': {'start': '2013-11-25T11:15:00Z',
    'end': '2020-01-20T21:45:00Z'}}},
 'MMA_SHA_2F-Secondary': {'description': ('[Fast-Track Product]: Secondary (inter

E.g. get the list of all the available models without their details:

In [23]:
request.available_models(details=False)

['IGRF12',
 'LCS-1',
 'MF7',
 'CHAOS-Core',
 'CHAOS-Static',
 'CHAOS-MMA-Primary',
 'CHAOS-MMA-Secondary',
 'CHAOS-6-Core',
 'CHAOS-6-Static',
 'CHAOS-6-MMA-Primary',
 'CHAOS-6-MMA-Secondary',
 'MCO_SHA_2C',
 'MCO_SHA_2D',
 'MLI_SHA_2C',
 'MLI_SHA_2D',
 'MMA_SHA_2C-Primary',
 'MMA_SHA_2C-Secondary',
 'MMA_SHA_2F-Primary',
 'MMA_SHA_2F-Secondary',
 'MIO_SHA_2C-Primary',
 'MIO_SHA_2C-Secondary',
 'MIO_SHA_2D-Primary',
 'MIO_SHA_2D-Secondary']

[TOP](#top)

<a id="SwarmRequest.get_model_info"/>

### Get information about one or more models.

It is possible to get information about one or more specific models using the *get_model_info* method.

In [31]:
# Import SwarmClient if not already done
from viresclient import SwarmRequest

# Get info on SwarmRequest.get_model_info
SwarmRequest.get_model_info?

Signature:
SwarmRequest.get_model_info(
    self,
    models=None,
    custom_model=None,
    original_response=False,
)
Docstring:
Get model info from server.

Handles the same models input as .set_products(), and returns a dict
like:

{'IGRF12': {
'expression': 'IGRF12(max_degree=13,min_degree=0)',
'validity': {'start': '1900-01-01T00:00:00Z', 'end': '2020-01-01T00:00:00Z'
}, ...}

If original_response=True, return the list of dicts like:

{'expression': 'MCO_SHA_2C(max_degree=16,min_degree=0)',
'name': 'MCO_SHA_2C',
'validity': {'start': '2013-11-30T14:38:24Z',
'end': '2018-01-01T00:00:00Z'}}, ...

Args:
    models (list/dict): as with set_products
    custom_model (str): as with set_products
    original_response (bool)

Returns:
    dict or list
File:      /opt/conda/lib/python3.7/site-packages/viresclient/_client_swarm.py
Type:      function


This method accepts the following arguments:

- **models** (*list[str]*, optional): models as a list of strings. If the list is not provided, returns information about all the available magnetico models.
- **custom_model** (*str*, optional): name of the file containing the spherical harmonics coefficients (custom model).
- **original_response** (*bool*, optional): if set to `False` (default), returns the result as a dictionary. If set to `True`, returns the result as a list of dictionaries.

E.g. get info about all the available magnetic models:

In [32]:
# Create the request object
request = SwarmRequest()

# Get model info
request.get_model_info()

{'MLI_SHA_2D': {'sources': ['SW_OPER_MLI_SHA_2D_00000000T000000_99999999T999999_0501'],
  'expression': 'MLI_SHA_2D(max_degree=133,min_degree=16)',
  'validity': {'start': '0001-01-01T00:00:00Z',
   'end': '4000-01-01T00:00:00Z'}},
 'MLI_SHA_2C': {'sources': ['SW_OPER_MLI_SHA_2C_00000000T000000_99999999T999999_0501'],
  'expression': 'MLI_SHA_2C(max_degree=120,min_degree=16)',
  'validity': {'start': '0001-01-01T00:00:00Z',
   'end': '4000-01-01T00:00:00Z'}},
 'CHAOS-Core': {'sources': ['SW_OPER_MCO_SHA_2X_19970101T000000_20200419T235959_0701'],
  'expression': "'CHAOS-Core'(max_degree=20,min_degree=1)",
  'validity': {'start': '1997-02-07T05:31:12.000003Z',
   'end': '2020-04-21T02:49:20.639999Z'}},
 'MIO_SHA_2C-Primary': {'sources': ['SW_OPER_MIO_SHA_2C_00000000T000000_99999999T999999_0501'],
  'expression': "'MIO_SHA_2C-Primary'(max_degree=60,min_degree=1)",
  'validity': {'start': '0001-01-01T00:00:00Z',
   'end': '4000-01-01T00:00:00Z'}},
 'LCS-1': {'sources': ['LCS-1.shc'],
  'ex

E.g. get info about CHAOS-Core and CHAOS-Static:

In [27]:
request.get_model_info(models=['CHAOS-Core', 'CHAOS-Static'])

{'CHAOS-Core': {'sources': ['SW_OPER_MCO_SHA_2X_19970101T000000_20200419T235959_0701'],
  'expression': "'CHAOS-Core'(max_degree=20,min_degree=1)",
  'validity': {'start': '1997-02-07T05:31:12.000003Z',
   'end': '2020-04-21T02:49:20.639999Z'}},
 'CHAOS-Static': {'sources': ['CHAOS-7_static.shc'],
  'expression': "'CHAOS-Static'(max_degree=185,min_degree=21)",
  'validity': {'start': '0001-01-01T00:00:00Z',
   'end': '4000-01-01T00:00:00Z'}}}

E.g. set `original_response=True` to get the result as a list of dictionaries:

In [28]:
request.get_model_info(models=['CHAOS-Core', 'CHAOS-Static'], original_response=True)

[{'sources': ['SW_OPER_MCO_SHA_2X_19970101T000000_20200419T235959_0701'],
  'expression': "'CHAOS-Core'(max_degree=20,min_degree=1)",
  'name': 'CHAOS-Core',
  'validity': {'start': '1997-02-07T05:31:12.000003Z',
   'end': '2020-04-21T02:49:20.639999Z'}},
 {'sources': ['CHAOS-7_static.shc'],
  'expression': "'CHAOS-Static'(max_degree=185,min_degree=21)",
  'name': 'CHAOS-Static',
  'validity': {'start': '0001-01-01T00:00:00Z',
   'end': '4000-01-01T00:00:00Z'}}]

E.g. get info on a custom model by providing the name of the file containing its coefficients:

In [30]:
# Download shc file with WGET (command line tool)
!wget "http://www.spacecenter.dk/files/magnetic-models/LCS-1/LCS-1.shc"

# Upload a .shc file and update the file name
request.get_model_info(custom_model='LCS-1.shc')

--2020-02-03 11:35:03--  http://www.spacecenter.dk/files/magnetic-models/LCS-1/LCS-1.shc
Resolving www.spacecenter.dk (www.spacecenter.dk)... 130.226.216.199
Connecting to www.spacecenter.dk (www.spacecenter.dk)|130.226.216.199|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 657666 (642K) [text/plain]
Saving to: ‘LCS-1.shc.1’

LCS-1.shc.1         100%[===================>] 642.25K  2.24MB/s    in 0.3s    

2020-02-03 11:35:04 (2.24 MB/s) - ‘LCS-1.shc.1’ saved [657666/657666]



{'Custom_Model': {'sources': [],
  'expression': 'Custom_Model(max_degree=185,min_degree=1)',
  'validity': {'start': '0001-01-01T00:00:00Z',
   'end': '4000-01-01T00:00:00Z'}}}

In [31]:
# Delete downloaded file(s)
!rm LCS-1.shc*

[TOP](#top)

<a id="SwarmRequest.get_orbit_number"/>

### Get the orbit number

It is possible to get the orbit number of a given spacecraft providing date and time.

In [33]:
# Import SwarmClient if not already done
from viresclient import SwarmRequest

# Get info on SwarmRequest.get_orbit_number
SwarmRequest.get_orbit_number?

Signature: SwarmRequest.get_orbit_number(self, spacecraft, input_time)
Docstring:
Translate a time to an orbit number.

Args:
    spacecraft (str): one of ('A','B','C') or
                        ("Alpha", "Bravo", "Charlie")
    input_time (datetime): a point in time

Returns:
    int: The current orbit number at the input_time
File:      /opt/conda/lib/python3.7/site-packages/viresclient/_client_swarm.py
Type:      function


Arguments:

- **spacecraft** (*str*): spacecraft identifier: `A`, `B` or `C`.
- **input_time** (*datetime.datetime*): date and time.

*input_time* must be provided as *datetime.datetime* object, thus we need to import the *datetime* class from the *datetime* package:

In [8]:
from datetime import datetime

E.g. get orbit numbers corresponding to date 2020-01-01 00:00:00 for the three spacecrafts:

In [34]:
# Create the request object
request = SwarmRequest()

# Create datetime object
dt = datetime(2020, 1, 1)

# Get results
for sc in ('A', 'B', 'C'):
    orbit = request.get_orbit_number(sc, dt)
    print(f's/c {sc}: {orbit}')

s/c A: 34324
s/c B: 33892
s/c C: 34320


[TOP](#top)

<a id="SwarmRequest.get_times_for_orbits"/>

### Get times for orbits

Get the time interval corresponding to a pair of orbit numbers using the *get_times_for_orbits* method.

In [35]:
# Import SwarmClient if not already done
from viresclient import SwarmRequest

# Get info on SwarmRequest.get_times_for_orbits
SwarmRequest.get_times_for_orbits?

Signature: SwarmRequest.get_times_for_orbits(self, spacecraft, start_orbit, end_orbit)
Docstring:
Translate a pair of orbit numbers to a time interval.

Args:
    spacecraft (str): one of ('A','B','C') or
                        ("Alpha", "Bravo", "Charlie")
    start_orbit (int): a starting orbit number
    end_orbit (int): a later orbit number

Returns:
    tuple (datetime): (start_time, end_time) The start time of the
    start_orbit and the ending time of the end_orbit.
    (Based on ascending nodes of the orbits)
File:      /opt/conda/lib/python3.7/site-packages/viresclient/_client_swarm.py
Type:      function


Arguments are:

- **spacecraft** (*str*): spacecraft identifier: `A`, `B` or `C`.
- **start_orbit** (*int*): start orbit number
- **end_orbit** (*int*): end orbit number

E.g. get time intervals corresponding to *start_orbit* 1000 and *end_orbit* 2000 for the three spacecrafts:

In [36]:
# Set start/end orbits
start_orbit = 1000
end_orbit = 2000

# Create the request object
request = SwarmRequest()

# Get results
for sc in ('A', 'B', 'C'):
    start_date, end_date = request.get_times_for_orbits(sc, start_orbit, end_orbit)
    print(f's/c {sc}: {start_date} - {end_date}')

s/c A: 2014-01-27 04:05:04.228591 - 2014-04-02 13:51:01.876037
s/c B: 2014-01-27 04:04:17.158156 - 2014-04-03 00:46:19.552254
s/c C: 2014-01-27 04:08:31.126684 - 2014-04-02 19:45:08.511104


[TOP](#top)

<a id="SwarmRequest.set_collection"/>

### Set collections

Before sending the request to the server, we need to set the Swarm collection including the measurement(s) of interest. This can be done using the *set_collection* method.

In [37]:
# Import SwarmClient if not already done
from viresclient import SwarmRequest

# Get info on SwarmRequest.set_collection
SwarmRequest.set_collection?

Signature: SwarmRequest.set_collection(self, *args)
Docstring:
Set the collection(s) to use.

Args:
    (str): one or several from .available_collections()
File:      /opt/conda/lib/python3.7/site-packages/viresclient/_client_swarm.py
Type:      function


The method accepts one or more collections provided as strings.

<a id="SwarmRequest.set_collection.example"/>

E.g. to get data from `SW_OPER_MAGA_LR_1B` and `SW_OPER_EFIA_LP_1B` collections:

In [38]:
# Create the request object
request = SwarmRequest()

# Set collections
request.set_collection('SW_OPER_MAGA_LR_1B', 'SW_OPER_EFIA_LP_1B')

[TOP](#top)

<a id="SwarmRequest.set_products"/>

### Set products

After setting the collection, it is possible to set the combination of measurements and/or auxiliaries and/or magnetic model(s) data to retrieve. This can be done using the *set_products* method.

In [39]:
# Import SwarmClient if not already done
from viresclient import SwarmRequest

# Get info on SwarmRequest.set_products
SwarmRequest.set_products?

Signature:
SwarmRequest.set_products(
    self,
    measurements=None,
    models=None,
    custom_model=None,
    auxiliaries=None,
    residuals=False,
    sampling_step=None,
)
Docstring:
Set the combination of products to retrieve.

If residuals=True then just get the measurement-model residuals,
otherwise get both measurement and model values.

Args:
    measurements (list(str)): from .available_measurements(collection_key)
    models (list(str)/dict): from .available_models() or defineable with custom expressions
    custom_model (str): path to a custom model in .shc format
    auxiliaries (list(str)): from .available_auxiliaries()
    residuals (bool): True if only returning measurement-model residual
    sampling_step (str): ISO_8601 duration, e.g. 10 seconds: PT10S, 1 minute: PT1M
File:      /opt/conda/lib/python3.7/site-packages/viresclient/_client_swarm.py
Type:      function


The following arguments are admitted:

- **measurements** (*list[str]*, optional): list of measurements to be downloaded. To get the list of the available measurements see [Available measurements](#SwarmRequest.available_measurements) (e.g.: `['F', 'B_NEC', 'Ne']`.).
- **models** (*list[str]*, optional): list of magnetic models. To get the list of the available models see [Available models](#SwarmRequest.available_models) (e.g.: `['CHAOS-Core', 'CHAOS-Static']`). In addition to the list, this parameters accepts also expression for the definition of magnetic models (e.g.: `'CHAOS = "CHAOS-Core" + "CHAOS-Static"'`).
- **custom_model** (*str*, optional): path to the the file containing the spherical harmonics coefficients of the custom model.
- **auxiliaries** (*list[str]*, optional): list of auxiliaries to be downloaded. To get the list of the available auxiliaries see [Available auxiliaries](#SwarmRequest.available_auxiliaries). Please note that the following parameters are always retrieved (i.e. they don't need to be specified): `Spacecraft`, `Timestamp`, `Latitude`, `Longitude`, `Radius`.
- **residuals** (*bool*, optional): if it is set to `True`, returns the residuals between measurements (specified with *measurements*) and models (specified with *models*). If it is set to `False` (default), returns measurements and models.
- **sampling_step** (*str*, optional): set the sampling step as an [ISO 8601 time interval](https://en.wikipedia.org/wiki/ISO_8601#Time_intervals). If not provided, data is returned with the original sampling.

E.g. get measurements: `F` and `B_NEC` from `SW_OPER_MAGA_LR_1B` and `Ne` from `SW_OPER_EFIA_LP_1B`, model `CHAOS = "CHAOS-Core" + "Chaos-Static"` and auxiliary `OrbitNumber` with a sampling step of 10 seconds.

In [41]:
# Create the request object
request = SwarmRequest()

# Set collections
request.set_collection('SW_OPER_MAGA_LR_1B', 'SW_OPER_EFIA_LP_1B')

# Set products
request.set_products(
    measurements=['F', 'B_NEC', 'Ne'],
    models=['CHAOS = "CHAOS-Core" + "CHAOS-Static"'],
    auxiliaries=['OrbitNumber'],
    residuals=False,
    sampling_step='PT10S'
)

[TOP](#top)

<a id="SwarmRequest.set_range_filter"/>

### Set/clear filters

Filter(s) can be applied to the requested measurements using the *set_range_filter* method.

In [42]:
# Import SwarmClient if not already done
from viresclient import SwarmRequest

# Get info on SwarmRequest.set_range_filter
SwarmRequest.set_range_filter?

Signature:
SwarmRequest.set_range_filter(
    self,
    parameter=None,
    minimum=None,
    maximum=None,
)
Docstring:
Set a filter to apply.

Filters data for minimum ≤ parameter ≤ maximum

Note:
    Apply multiple filters with successive calls to set_range_filter()

Args:
    parameter (str)
    minimum (float)
    maximum (float)
File:      /opt/conda/lib/python3.7/site-packages/viresclient/_client_swarm.py
Type:      function


This method allows to set filter(s) in the form: $min \le parameter \le max$.

Arguments:

- **parameter** (*str*, optional): parameter to be used as a filter (e.g. `Latitude`)
- **minimum** (*float*, optional): allowed minimum value
- **maximum** (*float*, optional): allowed maximum value

It is possible to apply multiple filters with consecutive calls to this method. E.g. to set filter: -20 <= `Longitude` <= 50 and 30 <= `Latitude` <= 70:

In [44]:
# Create the request object
request = SwarmRequest()

# Set collection
request.set_collection('SW_OPER_MAGA_LR_1B')

# Set product
request.set_products(measurements=['F', 'B_NEC'])

# Set filters
request.set_range_filter('Longitude', -20.0, 50.0)
request.set_range_filter('Latitude', 30.0, 70.0)

Filters can be removed using the *clear_range_filter* method:

In [45]:
request.clear_range_filter()

[TOP](#top)

<a id="SwarmRequest.get_between"/>

### Send request to the server

The request is sent to the server using the *get_between* method. 

In [2]:
# Import SwarmClient if not already done
from viresclient import SwarmRequest

# Get info on SwarmRequest.get_between
SwarmRequest.get_between?

Signature:
SwarmRequest.get_between(
    self,
    start_time=None,
    end_time=None,
    filetype='cdf',
    asynchronous=True,
    show_progress=True,
    nrecords_limit=None,
    tmpdir=None,
)
Docstring:
Make the server request and download the data.

Args:
    start_time (datetime / ISO_8601 string)
    end_time (datetime / ISO_8601 string)
    filetype (str): one of ('csv', 'cdf')
    asynchronous (bool): True for asynchronous processing,
        False for synchronous
    show_progress (bool): Set to False to remove progress bars
    nrecords_limit (int): Override the default limit per request
        (e.g. nrecords_limit=3456000)
    tmpdir (str): Override the default temporary file directory

Returns:
    ReturnedData:
File:      /opt/conda/lib/python3.7/site-packages/viresclient/_client.py
Type:      function


Arguments:

- **start_time** (*datetime.datetime* or *str*, optional): lower bound of temporal interval. If provided as string, it must be compliant to [ISO-8601](https://en.wikipedia.org/wiki/ISO_8601).
- **end_time** (*datetime.datetime* or *str*, optional): upper bound of temporal interval. If provided as string, it must be compliant to [ISO-8601](https://en.wikipedia.org/wiki/ISO_8601).
- **filetype** (*str*, optional): file format. Allowed values: `csv` and `cdf` (default).
- **asynchronous** (*bool*, optional): if `True` (default), set the asynchronous processing.
- **show_progress** (*bool*, optional): if `True` (default), enable the progress bar while processing and downloading data.
- **nrecords_limit** (*int*): overrides the limit of 3456000 records.

E.g. get measurements: F and B_NEC from SW_OPER_MAGA_LR_1B and Ne from SW_OPER_EFIA_LP_1B, model CHAOS = "CHAOS-Core" + "Chaos-Static" and auxiliary OrbitNumber with a sampling step of 10 seconds, between 2019-10-01T00:00:00 and 2019-10-01T01:00:00 in CDF format:

In [3]:
# Create the request object
request = SwarmRequest()

# Set collections
request.set_collection('SW_OPER_MAGA_LR_1B', 'SW_OPER_EFIA_LP_1B')

# Set products
request.set_products(
    measurements=['F', 'B_NEC', 'Ne'],
    models=['CHAOS = "CHAOS-Core" + "CHAOS-Static"'],
    auxiliaries=['OrbitNumber'],
    residuals=False,
    sampling_step='PT10S'
)

# Get data
data = request.get_between('2019-10-01T00:00:00', '2019-10-01T01:00:00')

[1/1] Processing:  100%|██████████|  [ Elapsed: 00:01, Remaining: 00:00 ]
      Downloading: 100%|██████████|  [ Elapsed: 00:00, Remaining: 00:00 ] (0.098MB)


Data is returned as `ReturnedData` object:

In [4]:
type(data)

viresclient._data_handling.ReturnedData

[TOP](#top)

<a id="ReturnedData"/>

## `ReturnedData`

This object holds data returned from the server (i.e. after invoking [`SwarmRequest.get_between()`](#SwarmRequest.get_between)).

In [40]:
from viresclient import ReturnedData

ReturnedData?

Init signature: ReturnedData(filetype=None, N=1, tmpdir=None)
Docstring:     
Flexible object for working with data returned from the server

Holds a list of ReturnedDataFile objects.
The number of them, N, is set upon initialisation.
Access the ReturnedDataFile objects directly via the list in ReturnedData.contents.

Example usage::

    ...
    data = request.get_between(..., ...)
    data.as_xarray()
    data.as_dataframe(expand=True)
    data.to_file()
File:           /opt/conda/lib/python3.7/site-packages/viresclient/_data_handling.py
Type:           type
Subclasses:     


[TOP](#top)

<a id="ReturnedData.sources"/>

### `ReturnedData.sources`

This attribute contains the list of source data from which the values have been extracted.

In [41]:
ReturnedData.sources?

Type:        property
String form: <property object at 0x7fe3fde024f8>
Docstring:  
Get list of source product identifiers.
        


E.g. to list the source of data retrieved in the `SwarmRequest.get_between()` [example](#SwarmRequest.get_between.example1):

In [42]:
data.sources

['CHAOS-7_static.shc',
 'SW_OPER_AUXAORBCNT_20131122T000000_20200128T000000_0001',
 'SW_OPER_EFIA_LP_1B_20190930T000000_20190930T235959_0501_MDR_EFI_LP',
 'SW_OPER_EFIA_LP_1B_20191001T000000_20191001T235959_0501_MDR_EFI_LP',
 'SW_OPER_MAGA_LR_1B_20191001T000000_20191001T235959_0505_MDR_MAG_LR',
 'SW_OPER_MCO_SHA_2X_19970101T000000_20200419T235959_0701']

[TOP](#top)

<a id="ReturnedData.contents"/>

### `ReturnedData.contents`

This attribute contains the list of `ReturnedDataFile` objects downloaded from the server.

In [43]:
ReturnedData.contents?

Type:        property
String form: <property object at 0x7fe3fddbb7c8>
Docstring:  
List of ReturnedDataFile objects
        


E.g. get the list of `ReturnedDataFile` objects downloaded in the `SwarmRequest.get_between()` [example](#SwarmRequest.get_between.example1):

In [44]:
data.contents

[TOP](#top)

<a id="ReturnedData.filetype"/>

### `ReturnedData.filetype`

This attribute contains the type of downloaded files (i.e. `cdf` or `csv`)

In [45]:
ReturnedData.filetype?

Type:        property
String form: <property object at 0x7fe3fde02548>
Docstring:  
Filetype string
        


E.g. get type of file downloaded in the `SwarmRequest.get_between()` [example](#SwarmRequest.get_between.example1):

In [46]:
data.filetype

'cdf'

[TOP](#top)

<a id="ReturnedData.magnetic_models"/>

### `ReturnedData.magnetic_models`

This attribute contains the list of the magnetic models.

In [47]:
ReturnedData.magnetic_models?

Type:        property
String form: <property object at 0x7fe3fddbb6d8>
Docstring:  
Get list of magnetic models used.
        


E.g. with reference to the last downloaded data:

In [48]:
data.magnetic_models

["CHAOS = 'CHAOS-Core'(max_degree=20,min_degree=1) + 'CHAOS-Static'(max_degree=185,min_degree=21)"]

[TOP](#top)

<a id="ReturnedData.range_filters"/>

### `ReturnedData.range_filters`

This attribute contains the list of applied filters:

In [49]:
ReturnedData.range_filters?

Type:        property
String form: <property object at 0x7fe3fddbb728>
Docstring:  
Get list of filters applied.
        


E.g. with reference to the last downloaded data:

In [50]:
data.range_filters

[]

[TOP](#top)

<a id="ReturnedData.as_dataframe"/>

### `ReturnedData.as_dataframe()`

Convert `ReturnedData` object to pandas [`DataFrame`](https://pandas.pydata.org/pandas-docs/stable/reference/frame.html) (see pandas reference documentation at  https://pandas.pydata.org/pandas-docs/stable/index.html).

In [51]:
ReturnedData.as_dataframe?

Signature: ReturnedData.as_dataframe(self, expand=False)
Docstring:
Convert the data to a pandas DataFrame.

If expand is True, expand some columns, e.g.:

B_NEC -> B_NEC_N, B_NEC_E, B_NEC_C

B_VFM -> B_VFM_i, B_VFM_j, B_VFM_k

Args:
    expand (bool)

Returns:
    pandas.DataFrame
File:      /opt/conda/lib/python3.7/site-packages/viresclient/_data_handling.py
Type:      function


The optional `expand` parameter is, by default, set to `False`. This means that the vector parameters are presented as arrays (i.e. all the vector components in a single column). E.g. with reference to data returned in `SwarmRequest.get_between()` [example](#SwarmRequest.get_between.example1):

In [52]:
data.as_dataframe().head()

,Spacecraft,Latitude,Longitude,Radius,F,F_CHAOS,B_NEC,B_NEC_CHAOS,Ne,OrbitNumber
2019-10-01 00:00:00,A,-33.130184,37.641386,6820061.65,26246.0328,26240.985891,"[11030.816, -6222.3955000000005, -22988.693300...","[11046.424869499242, -6221.021024854402, -2297...",108830.9,32904
2019-10-01 00:00:01,A,-33.194279,37.641441,6820066.90,26249.4370,26244.351869,"[11018.4866, -6234.7621, -22995.146200000003]","[11034.0388145068, -6233.415904174271, -22981....",108985.6,32904
2019-10-01 00:00:02,A,-33.258373,37.641502,6820072.14,26252.8804,26247.760366,"[11006.2309, -6247.1441, -23001.5979]","[11021.728601082996, -6245.801052536994, -2298...",110149.9,32904
2019-10-01 00:00:03,A,-33.322467,37.641570,6820077.38,26256.3718,26251.211496,"[10994.072400000001, -6259.4914, -23008.0711]","[11009.494017179964, -6258.176491021181, -2299...",110497.3,32904
2019-10-01 00:00:04,A,-33.386560,37.641644,6820082.60,26259.9015,26254.705565,"[10981.9362, -6271.791200000001, -23014.524800...","[10997.334942448828, -6270.542276417115, -2300...",111010.1,32904


Setting `expand=True`, the vector parameters are expanded with each componet in a different column. E.g.:

In [53]:
data.as_dataframe(expand=True).head()

,Spacecraft,Latitude,Longitude,Radius,F,F_CHAOS,B_NEC_N,B_NEC_E,B_NEC_C,B_NEC_CHAOS_N,B_NEC_CHAOS_E,B_NEC_CHAOS_C,Ne,OrbitNumber
2019-10-01 00:00:00,A,-33.130184,37.641386,6820061.65,26246.0328,26240.985891,11030.8160,-6222.3955,-22988.6933,11046.424869,-6221.021025,-22975.307083,108830.9,32904
2019-10-01 00:00:01,A,-33.194279,37.641441,6820066.90,26249.4370,26244.351869,11018.4866,-6234.7621,-22995.1462,11034.038815,-6233.415904,-22981.743159,108985.6,32904
2019-10-01 00:00:02,A,-33.258373,37.641502,6820072.14,26252.8804,26247.760366,11006.2309,-6247.1441,-23001.5979,11021.728601,-6245.801053,-22988.179399,110149.9,32904
2019-10-01 00:00:03,A,-33.322467,37.641570,6820077.38,26256.3718,26251.211496,10994.0724,-6259.4914,-23008.0711,11009.494017,-6258.176491,-22994.616185,110497.3,32904
2019-10-01 00:00:04,A,-33.386560,37.641644,6820082.60,26259.9015,26254.705565,10981.9362,-6271.7912,-23014.5248,10997.334942,-6270.542276,-23001.054063,111010.1,32904


[TOP](#top)

<a id="ReturnedData.as_xarray"/>

### `ReturnedData.as_xarray()`

Convert `ReturnedData` object to xarray [`Dataset`](http://xarray.pydata.org/en/stable/data-structures.html#dataset) (see xarray reference documentation at http://xarray.pydata.org/en/stable/index.html).

In [54]:
ReturnedData.as_xarray?

Signature: ReturnedData.as_xarray(self)
Docstring:
Convert the data to an xarray Dataset.

Returns:
    xarray.Dataset
File:      /opt/conda/lib/python3.7/site-packages/viresclient/_data_handling.py
Type:      function


E.g. with reference to data returned in `SwarmRequest.get_between()` [example](#SwarmRequest.get_between.example1):

In [55]:
data.as_xarray()

<xarray.Dataset>
Dimensions:      (NEC: 3, Timestamp: 3600)
Coordinates:
  * Timestamp    (Timestamp) datetime64[ns] 2019-10-01 ... 2019-10-01T00:59:59
Dimensions without coordinates: NEC
Data variables:
    Spacecraft   (Timestamp) <U1 'A' 'A' 'A' 'A' 'A' 'A' ... 'A' 'A' 'A' 'A' 'A'
    Latitude     (Timestamp) float64 -33.13 -33.19 -33.26 ... 83.7 83.76 83.81
    Longitude    (Timestamp) float64 37.64 37.64 37.64 ... -134.3 -134.1 -133.9
    Radius       (Timestamp) float64 6.82e+06 6.82e+06 ... 6.803e+06 6.803e+06
    F            (Timestamp) float64 2.625e+04 2.625e+04 ... 4.744e+04 4.743e+04
    F_CHAOS      (Timestamp) float64 2.624e+04 2.624e+04 ... 4.743e+04 4.743e+04
    B_NEC        (Timestamp, NEC) float64 1.103e+04 -6.222e+03 ... 4.742e+04
    B_NEC_CHAOS  (Timestamp, NEC) float64 1.105e+04 -6.221e+03 ... 4.742e+04
    Ne           (Timestamp) float64 1.088e+05 1.09e+05 ... 3.016e+04 3.059e+04
    OrbitNumber  (Timestamp) int32 32904 32904 32904 32904 ... 32905 32905 32905
Attributes:
    Sources:         ['CHAOS-7_static.shc', 'SW_OPER_AUXAORBCNT_20131122T0000...
    MagneticModels:  ["CHAOS = 'CHAOS-Core'(max_degree=20,min_degree=1) + 'CH...
    RangeFilters:    []

[TOP](#top)

<a id="ReturnedData.to_file"/>

### `ReturnedData.to_file()`

Save `ReturnedData` object to file.

In [56]:
ReturnedData.to_file?

Signature: ReturnedData.to_file(self, path, overwrite=False)
Docstring:
Saves the data to the specified file, when data is only in one file.

Only write to file if it does not yet exist, or if overwrite=True.
Currently handles CSV and CDF formats.

Args:
    path (str): path to the file to save as
    overwrite (bool): Will overwrite existing file if True
File:      /opt/conda/lib/python3.7/site-packages/viresclient/_data_handling.py
Type:      function


The `path` parameter indicates the output file path.

The `overwrite` parameter (default: `False`) allows to overwrite the file if already present at `path`.

<a id="ReturnedData.to_file"/>

E.g.:

In [57]:
data.to_file('out.cdf', True)

Data written to out.cdf


Try executing again the above cell after setting `overwrite` to `False`.

**Note**: this method can be used only if the amount of downloaded data is small (i.e. if the request is not split between multiple requests).

<a id="ReturnedData.to_file.example1"/>

E.g.: try to download `U_orbit`, `Ne`, `Te`, `Vs` EFI data between 2019-10-01T00:00:00' and '2019-11-01T00:00:00':

In [58]:
newreq = SwarmRequest()
newreq.set_collection('SW_OPER_EFIA_LP_1B')
newreq.set_products(measurements=['U_orbit', 'Ne', 'Te', 'Vs'])
hugedata = newreq.get_between('2019-10-01T00:00:00', '2019-11-01T00:00:00')

[1/2] Processing:  100%|██████████|  [ Elapsed: 00:03, Remaining: 00:00 ]
      Downloading: 100%|██████████|  [ Elapsed: 00:02, Remaining: 00:00 ] (279.743MB)
[2/2] Processing:  100%|██████████|  [ Elapsed: 00:01, Remaining: 00:00 ]
      Downloading: 100%|██████████|  [ Elapsed: 00:00, Remaining: 00:00 ] (67.389MB)


The request is split between multiple requests. Try to execute the cell below:

In [ ]:
hugedata.to_file('huge.cdf', True)

In [60]:
!rm *.cdf

[TOP](#top)

<a id="ReturnedData.to_files"/>

### `ReturnedData.to_files()`

Save the `ReturnedData` object to file(s).

In [61]:
ReturnedData.to_files?

Signature: ReturnedData.to_files(self, paths, overwrite=False)
Docstring:
Saves the data to the specified files.

Only write to file if it does not yet exist, or if overwrite=True.
Currently handles CSV and CDF formats.

Args:
    paths (list of str): paths to the files to save as
    overwrite (bool): Will overwrite existing file if True
File:      /opt/conda/lib/python3.7/site-packages/viresclient/_data_handling.py
Type:      function


The `paths` parameter is a list of strings indicating the output file paths.

The `overwrite` parameter (default: `False`) allows to overwrite the file if already present.

In [62]:
data.to_files(['out.cdf'], True)

Data written to out.cdf


This method allows to write data to file(s) even if the request has been split between multiple requests. E.g. try to save `hugedata` to two files:

In [63]:
hugedata.to_files(['first.cdf', 'second.cdf'], True)

Data written to first.cdf
Data written to second.cdf


The number of files to be specified must be equal to the number of files indicated by `ReturnedData.contents`. In the above case:

In [64]:
hugedata.contents

In [65]:
!rm *.cdf

[TOP](#top)

<a id="ReturnedDataFile"/>

## `ReturnedDataFile`

This object holds the file downloaded from the server. Even if the file is not saved (with `ReturnedData.to_files()` method) it is saved into a temporary file and automatically deleted when not needed anymore.

In [66]:
from viresclient import ReturnedDataFile

In [67]:
ReturnedDataFile?

Init signature: ReturnedDataFile(filetype=None, tmpdir=None)
Docstring:     
For handling individual files returned from the server.

Holds the data returned from the server and the data type.
Data is held in a NamedTemporaryFile, which is automatically closed and
destroyed when it goes out of scope.
Provides output to different file types and data objects.
File:           /opt/conda/lib/python3.7/site-packages/viresclient/_data_handling.py
Type:           type
Subclasses:     


It is possible to access the `ReturnedDataFile` objects via the `ReturnedData.contents` attribute. E.g.:

In [68]:
data.contents

[TOP](#top)

<a id="ReturnedDataFile._file"/>

### `ReturnedDataFile._file`

This attribute returns the [`NamedTemporaryFile`](https://docs.python.org/3/library/tempfile.html#tempfile.NamedTemporaryFile) corresponding to the `ReturnedDataFile`. E.g.:

In [69]:
data.contents[0]._file

[TOP](#top)

<a id="ReturnedDataFile.filetype"/>

### `ReturnedDataFile.filetype`

This attribute contains the type of downloaded files (i.e. `cdf`, `csv` or `nc`)

In [70]:
ReturnedDataFile.filetype?

Type:        property
String form: <property object at 0x7fe4200409a8>
Docstring:  
Filetype is one of ("csv", "cdf", "nc")
        


E.g. for `data`:

In [71]:
data.contents[0].filetype

'cdf'

[TOP](#top)

<a id="ReturnedDataFile.as_dataframe"/>

### `ReturnedDataFile.as_dataframe()`

Convert `ReturnedDataFile` object to pandas [`DataFrame`](https://pandas.pydata.org/pandas-docs/stable/reference/frame.html) (see pandas reference documentation at  https://pandas.pydata.org/pandas-docs/stable/index.html).

In [72]:
ReturnedDataFile.as_dataframe?

Signature: ReturnedDataFile.as_dataframe(self, expand=False)
Docstring:
Convert the data to a pandas DataFrame.

Returns:
    pandas.DataFrame
File:      /opt/conda/lib/python3.7/site-packages/viresclient/_data_handling.py
Type:      function


The optional `expand` parameter is, by default, set to `False`. This means that the vector parameters are presented as arrays (i.e. all the vector components in a single column). E.g. with reference to data returned in `SwarmRequest.get_between()` [example](#SwarmRequest.get_between.example1):

In [73]:
data.contents[0].as_dataframe().head()

,Spacecraft,Latitude,Longitude,Radius,F,F_CHAOS,B_NEC,B_NEC_CHAOS,Ne,OrbitNumber
2019-10-01 00:00:00,A,-33.130184,37.641386,6820061.65,26246.0328,26240.985891,"[11030.816, -6222.3955000000005, -22988.693300...","[11046.424869499242, -6221.021024854402, -2297...",108830.9,32904
2019-10-01 00:00:01,A,-33.194279,37.641441,6820066.90,26249.4370,26244.351869,"[11018.4866, -6234.7621, -22995.146200000003]","[11034.0388145068, -6233.415904174271, -22981....",108985.6,32904
2019-10-01 00:00:02,A,-33.258373,37.641502,6820072.14,26252.8804,26247.760366,"[11006.2309, -6247.1441, -23001.5979]","[11021.728601082996, -6245.801052536994, -2298...",110149.9,32904
2019-10-01 00:00:03,A,-33.322467,37.641570,6820077.38,26256.3718,26251.211496,"[10994.072400000001, -6259.4914, -23008.0711]","[11009.494017179964, -6258.176491021181, -2299...",110497.3,32904
2019-10-01 00:00:04,A,-33.386560,37.641644,6820082.60,26259.9015,26254.705565,"[10981.9362, -6271.791200000001, -23014.524800...","[10997.334942448828, -6270.542276417115, -2300...",111010.1,32904


Setting `expand=True`, the vector parameters are expanded with each componet in a different column. E.g.:

In [74]:
data.contents[0].as_dataframe(expand=True).head()

,Spacecraft,Latitude,Longitude,Radius,F,F_CHAOS,B_NEC_N,B_NEC_E,B_NEC_C,B_NEC_CHAOS_N,B_NEC_CHAOS_E,B_NEC_CHAOS_C,Ne,OrbitNumber
2019-10-01 00:00:00,A,-33.130184,37.641386,6820061.65,26246.0328,26240.985891,11030.8160,-6222.3955,-22988.6933,11046.424869,-6221.021025,-22975.307083,108830.9,32904
2019-10-01 00:00:01,A,-33.194279,37.641441,6820066.90,26249.4370,26244.351869,11018.4866,-6234.7621,-22995.1462,11034.038815,-6233.415904,-22981.743159,108985.6,32904
2019-10-01 00:00:02,A,-33.258373,37.641502,6820072.14,26252.8804,26247.760366,11006.2309,-6247.1441,-23001.5979,11021.728601,-6245.801053,-22988.179399,110149.9,32904
2019-10-01 00:00:03,A,-33.322467,37.641570,6820077.38,26256.3718,26251.211496,10994.0724,-6259.4914,-23008.0711,11009.494017,-6258.176491,-22994.616185,110497.3,32904
2019-10-01 00:00:04,A,-33.386560,37.641644,6820082.60,26259.9015,26254.705565,10981.9362,-6271.7912,-23014.5248,10997.334942,-6270.542276,-23001.054063,111010.1,32904


[TOP](#top)

<a id="ReturnedDataFile.as_xarray"/>

### `ReturnedDataFile.as_xarray()`

Convert `ReturnedDataFile` object to xarray [`Dataset`](http://xarray.pydata.org/en/stable/data-structures.html#dataset) (see xarray reference documentation at http://xarray.pydata.org/en/stable/index.html).

In [75]:
ReturnedDataFile.as_xarray?

Signature: ReturnedDataFile.as_xarray(self, group=None)
Docstring:
Convert the data to an xarray Dataset.

Note:
    Does not support csv

    Only supports scalar and 3D vectors (currently)

Returns:
    xarray.Dataset
File:      /opt/conda/lib/python3.7/site-packages/viresclient/_data_handling.py
Type:      function


E.g. with reference to data returned in `SwarmRequest.get_between()` [example](#SwarmRequest.get_between.example1):

In [76]:
data.contents[0].as_xarray()

<xarray.Dataset>
Dimensions:      (NEC: 3, Timestamp: 3600)
Coordinates:
  * Timestamp    (Timestamp) datetime64[ns] 2019-10-01 ... 2019-10-01T00:59:59
Dimensions without coordinates: NEC
Data variables:
    Spacecraft   (Timestamp) <U1 'A' 'A' 'A' 'A' 'A' 'A' ... 'A' 'A' 'A' 'A' 'A'
    Latitude     (Timestamp) float64 -33.13 -33.19 -33.26 ... 83.7 83.76 83.81
    Longitude    (Timestamp) float64 37.64 37.64 37.64 ... -134.3 -134.1 -133.9
    Radius       (Timestamp) float64 6.82e+06 6.82e+06 ... 6.803e+06 6.803e+06
    F            (Timestamp) float64 2.625e+04 2.625e+04 ... 4.744e+04 4.743e+04
    F_CHAOS      (Timestamp) float64 2.624e+04 2.624e+04 ... 4.743e+04 4.743e+04
    B_NEC        (Timestamp, NEC) float64 1.103e+04 -6.222e+03 ... 4.742e+04
    B_NEC_CHAOS  (Timestamp, NEC) float64 1.105e+04 -6.221e+03 ... 4.742e+04
    Ne           (Timestamp) float64 1.088e+05 1.09e+05 ... 3.016e+04 3.059e+04
    OrbitNumber  (Timestamp) int32 32904 32904 32904 32904 ... 32905 32905 32905

[TOP](#top)

<a id="ReturnedDataFile.to_file"/>

### `ReturnedDataFile.to_file()`

Save `ReturnedDataFile` object to file.

In [77]:
ReturnedDataFile.to_file?

Signature: ReturnedDataFile.to_file(self, path, overwrite=False)
Docstring:
Saves the data to the specified file.

Only write to file if it does not yet exist, or if overwrite=True.
Currently handles CSV and CDF formats.

Args:
    path (str): path to the file to save as
    overwrite (bool): Will overwrite existing file if True
File:      /opt/conda/lib/python3.7/site-packages/viresclient/_data_handling.py
Type:      function


The `path` parameter indicates the output file path.

The `overwrite` parameter (default: `False`) allows to overwrite the file if already present at `path`.

In [78]:
data.contents[0].to_file('out.cdf', True)

Data written to out.cdf


Try executing again the above cell after setting `overwrite` to `False`.

In [79]:
!rm out.cdf

[TOP](#top)

<a id="ClientConfig"/>

## `ClientConfig`

Class for client configuration.

In [80]:
from viresclient import ClientConfig

ClientConfig?

Init signature: ClientConfig(path=None)
Docstring:     
Client configuration.

Example usage::

  cc = ClientConfig()      # use default configuration file
  cc = ClientConfig("./viresconf.ini")  # use custom configuration file

  print(cc.path)           # print path
  print(cc)                # print the whole configuration

  cc.default_url = "https://foo.bar/ows"  # set default server

  # access to credentials configuration ...
  cc.set_site_config("https://foo.bar/ows", username="...", password="...")
  cc.set_site_config("https://foo2.bar/ows", token="...")

  cc.save()    # save configuration
File:           /opt/conda/lib/python3.7/site-packages/viresclient/_config.py
Type:           type
Subclasses:     


Load `viresclient` configuration from the configuration file.

<a id="ClientConfig.example1"/>

The `path` argument is optional and is needed to specify the configuration file. If it is not specified, the configuration file will be `~/.viresclient.ini`. E.g.:

In [81]:
default = ClientConfig()

[TOP](#top)

<a id="ClientConfig.path"/>

### `ClientConfig.path`

Get path of the configuration file.

In [82]:
ClientConfig.path?

Type:        property
String form: <property object at 0x7fe4202774a8>
Docstring:   Get path of the configuration file. 


E.g. for default configuration file:

In [83]:
default.path

'/home/jovyan/.viresclient.ini'

[TOP](#top)

<a id="ClientConfig.default_url"/>

### `ClientConfig.default_url`

Get/set the default server URL (i.e. the one used when `SwarmClient` class is invoked without URL).

In [84]:
ClientConfig.default_url?

Type:        property
String form: <property object at 0x7fe420289ea8>
Docstring:   Get default URL or None if not set. 


E.g. for default configuration file:

In [85]:
default.default_url

'https://vires.services/ows'

If the default URL is not set, the attribute returns `None`. E.g:

In [86]:
# Create new configuration
newcfg = ClientConfig('newfile.ini')
type(newcfg.default_url)

NoneType

The attribute can be used also to set teh default URL, e.g.:

In [87]:
newcfg.default_url = 'https://vires.services/ows'

[TOP](#top)

<a id="ClientConfig.set_site_config"/>

### `ClientConfig.set_site_config()`

Set site configuration.

In [88]:
ClientConfig.set_site_config?

Signature: ClientConfig.set_site_config(self, url, **options)
Docstring: Set configuration for the given URL. 
File:      /opt/conda/lib/python3.7/site-packages/viresclient/_config.py
Type:      function


Arguments:
- `url`: server URL
- `**options`: options to be configured.

E.g.:

In [89]:
newcfg.set_site_config('https://vires.services/ows', token='mytokenstring')

[TOP](#top)

<a id="ClientConfig.get_site_config"/>

### `ClientConfig.get_site_config()`

Get site configuration as a dictionary.

In [90]:
ClientConfig.get_site_config?

Signature: ClientConfig.get_site_config(self, url)
Docstring: Get configuration for the given URL. 
File:      /opt/conda/lib/python3.7/site-packages/viresclient/_config.py
Type:      function


In [91]:
newcfg.get_site_config('https://vires.services/ows')

{'token': 'mytokenstring'}

[TOP](#top)

<a id="ClientConfig.save"/>

### `ClientConfig.save()`

Save configuration to `ClientConfig.path`.

In [92]:
ClientConfig.save?

Signature: ClientConfig.save(self)
Docstring: Save the configuration file. 
File:      /opt/conda/lib/python3.7/site-packages/viresclient/_config.py
Type:      function


E.g.:

In [93]:
newcfg.save()

Let's have a look to the new configuration file:

In [94]:
!cat newfile.ini

[default]
url = https://vires.services/ows

[https://vires.services/ows]
token = mytokenstring



In [95]:
del newcfg

In [96]:
!rm newfile.ini

[TOP](#top)

<a id="DataUpload"/>

## `DataUpload`

Class for the user data upload. Supported file format are described in: https://github.com/ESA-VirES/VirES-Server/blob/master/vires/custom_data_format_description.md.

In [97]:
from viresclient import DataUpload

DataUpload?

Init signature: DataUpload(url, token, **kwargs)
Docstring:     
VirES for Swarm data upload API proxy.

Example usage::

  from viresclient import ClientConfig, DataUpload

  du = DataUpload("https://vires.services", token="...")

  cc = ClientConfig()
  url = cc.default_url
  du = DataUpload(url, **cc.get_site_config(url))

  # upload file
  info = du.post("example.csv")
  print(info)

  # get information about the uploaded files
  info = du.get()
  print(info)

  # remove any uploaded files
  du.clear()

  # check if the upload is valid and get list of missing mandatory parameters
  info = du.post("example.cdf")
  is_valid = info.get('is_valid', True)
  missing_fields = info.get('missing_fields', {}).keys()
  print(is_valid, missing_fields)

  # get constant parameters
  id = info['identifier']
  parameters = du.get_constant_parameters(id)
  print(parameters)

  # set new constant parameters
  parameters = du.set_constant_parameters(id, {'Radius': 7000000, 'Latitude': 24.0})
  print

To create the `DataUpload` object we must provide the server URL (via the `url` parameter) and the access token (via the `token` parameter). Since these parameters are set in the configuration file, we can retrieve them using the `ClientConfig` object created in `ClientConfig` [example](#ClientConfig.example1):

In [98]:
url = default.default_url
token = default.get_site_config(url)['token']

du = DataUpload(url, token)

[TOP](#top)

<a id="DataUpload.post"/>

### `DataUpload.post()`

Upload file to the server.

In [99]:
DataUpload.post?

Signature: DataUpload.post(self, file, filename=None)
Docstring:
HTTP POST multipart/form-data
Upload file to the server and get info about the uploaded file.
File:      /opt/conda/lib/python3.7/site-packages/viresclient/_api/upload.py
Type:      function


E.g. upload the product previously downloaded in `ReturnedData.to_file()` [example](#ReturnedData.to_file.example1).

In [100]:
data.to_file('out.cdf', True)
info = du.post('out.cdf')

Data written to out.cdf


In [101]:
info

{'missing_fields': {},
 'source_fields': ['OrbitNumber',
  'F',
  'Timestamp',
  'B_NEC_CHAOS',
  'F_CHAOS',
  'Ne',
  'Longitude',
  'B_NEC',
  'Radius',
  'Latitude'],
 'content_type': 'application/x-cdf',
 'owner': 'lmar76',
 'size': 404015,
 'end': '2019-10-01T00:59:59Z',
 'constant_fields': {},
 'data_file': 'out.cdf',
 'fields': {'F': {'shape': [], 'cdf_type': 45, 'data_type': 'CDF_DOUBLE'},
  'Timestamp': {'shape': [], 'cdf_type': 31, 'data_type': 'CDF_EPOCH'},
  'Latitude': {'shape': [], 'cdf_type': 45, 'data_type': 'CDF_DOUBLE'},
  'B_NEC_CHAOS': {'shape': [3], 'cdf_type': 45, 'data_type': 'CDF_DOUBLE'},
  'F_CHAOS': {'shape': [], 'cdf_type': 45, 'data_type': 'CDF_DOUBLE'},
  'Ne': {'shape': [], 'cdf_type': 45, 'data_type': 'CDF_DOUBLE'},
  'Longitude': {'shape': [], 'cdf_type': 45, 'data_type': 'CDF_DOUBLE'},
  'B_NEC': {'shape': [3], 'cdf_type': 45, 'data_type': 'CDF_DOUBLE'},
  'Radius': {'shape': [], 'cdf_type': 45, 'data_type': 'CDF_DOUBLE'},
  'OrbitNumber': {'shape': []

In [102]:
!rm out.cdf

[TOP](#top)

<a id="DataUpload.ids"/>

### `DataUpload.ids`

Attribute reporting the list of identifiers of the uploaded files.

In [103]:
DataUpload.ids?

Type:        property
String form: <property object at 0x7fe3fddd4a98>
Docstring:   Get list of identifiers. 


E.g.:

In [104]:
du.ids

['b34f0ca3-7b8f-40a6-b4ab-54f84eb45a50']

[TOP](#top)

<a id="DataUpload.get"/>

### `DataUpload.get()`

Get info on uploaded files.

In [105]:
DataUpload.get?

Signature: DataUpload.get(self, identifier=None)
Docstring:
REST/API GET
If an identifier provided, get info about the uploaded item.
If no identifier provided, list all uploaded items.
File:      /opt/conda/lib/python3.7/site-packages/viresclient/_api/upload.py
Type:      function


If the `identifier` is not provided, the method returns info about all the uploaded files as a list:

In [106]:
du.get()

[{'missing_fields': {},
  'source_fields': ['OrbitNumber',
   'F',
   'Timestamp',
   'B_NEC_CHAOS',
   'F_CHAOS',
   'Ne',
   'Longitude',
   'B_NEC',
   'Radius',
   'Latitude'],
  'content_type': 'application/x-cdf',
  'owner': 'lmar76',
  'size': 404015,
  'end': '2019-10-01T00:59:59Z',
  'constant_fields': {},
  'data_file': 'out.cdf',
  'fields': {'F': {'shape': [], 'cdf_type': 45, 'data_type': 'CDF_DOUBLE'},
   'Timestamp': {'shape': [], 'cdf_type': 31, 'data_type': 'CDF_EPOCH'},
   'Latitude': {'shape': [], 'cdf_type': 45, 'data_type': 'CDF_DOUBLE'},
   'B_NEC_CHAOS': {'shape': [3], 'cdf_type': 45, 'data_type': 'CDF_DOUBLE'},
   'F_CHAOS': {'shape': [], 'cdf_type': 45, 'data_type': 'CDF_DOUBLE'},
   'Ne': {'shape': [], 'cdf_type': 45, 'data_type': 'CDF_DOUBLE'},
   'Longitude': {'shape': [], 'cdf_type': 45, 'data_type': 'CDF_DOUBLE'},
   'B_NEC': {'shape': [3], 'cdf_type': 45, 'data_type': 'CDF_DOUBLE'},
   'Radius': {'shape': [], 'cdf_type': 45, 'data_type': 'CDF_DOUBLE'},
   

Providing the `identifier`, it returns info about the specified file:

In [107]:
du.get(du.ids[0])

{'missing_fields': {},
 'source_fields': ['OrbitNumber',
  'F',
  'Timestamp',
  'B_NEC_CHAOS',
  'F_CHAOS',
  'Ne',
  'Longitude',
  'B_NEC',
  'Radius',
  'Latitude'],
 'content_type': 'application/x-cdf',
 'owner': 'lmar76',
 'size': 404015,
 'end': '2019-10-01T00:59:59Z',
 'constant_fields': {},
 'data_file': 'out.cdf',
 'fields': {'F': {'shape': [], 'cdf_type': 45, 'data_type': 'CDF_DOUBLE'},
  'Timestamp': {'shape': [], 'cdf_type': 31, 'data_type': 'CDF_EPOCH'},
  'Latitude': {'shape': [], 'cdf_type': 45, 'data_type': 'CDF_DOUBLE'},
  'B_NEC_CHAOS': {'shape': [3], 'cdf_type': 45, 'data_type': 'CDF_DOUBLE'},
  'F_CHAOS': {'shape': [], 'cdf_type': 45, 'data_type': 'CDF_DOUBLE'},
  'Ne': {'shape': [], 'cdf_type': 45, 'data_type': 'CDF_DOUBLE'},
  'Longitude': {'shape': [], 'cdf_type': 45, 'data_type': 'CDF_DOUBLE'},
  'B_NEC': {'shape': [3], 'cdf_type': 45, 'data_type': 'CDF_DOUBLE'},
  'Radius': {'shape': [], 'cdf_type': 45, 'data_type': 'CDF_DOUBLE'},
  'OrbitNumber': {'shape': []

[TOP](#top)

<a id="DataUpload.set_constant_parameters"/>

### `DataUpload.set_constant_parameters()`

Set constant parameters to the uploaded product file.

In [108]:
du.set_constant_parameters?

Signature: du.set_constant_parameters(identifier, parameters, replace=False)
Docstring:
Set constant parameters form from give key value dictionary.
Set replace to True if you prefer to replace the already set parameters
rather then update them.
File:      /opt/conda/lib/python3.7/site-packages/viresclient/_api/upload.py
Type:      method


Arguments:
- `identifier`: file identifier
- `parameters`: constant parameters provided as ditionary
- `replace`: if set to `True`, all the parameters will be replaced by the new parameters, otherwise the new parameters will update the existing ones.

E.g.:

In [109]:
du.set_constant_parameters(du.ids[0], {'param1': 12345, 'param2': 34567})

{'param2': 34567, 'param1': 12345}

If we want to set `param1` to a new value we can update the existing parameters:

In [110]:
du.set_constant_parameters(du.ids[0], {'param1': 1})

{'param2': 34567, 'param1': 1}

or replace all the parameters:

In [111]:
du.set_constant_parameters(du.ids[0], {'param1': 1}, replace=True)

{'param1': 1}

Note that `param2` does not exist anymore.

[TOP](#top)

<a id="DataUpload.get_constant_parameters"/>

### `DataUpload.get_constant_parameters()`

Get the list of constant parameters.

In [112]:
DataUpload.get_constant_parameters?

Signature: DataUpload.get_constant_parameters(self, identifier)
Docstring: Get dictionary of the currently set constant parameters. 
File:      /opt/conda/lib/python3.7/site-packages/viresclient/_api/upload.py
Type:      function


E.g. get the list of constant parameters set in the previous example:

In [113]:
du.get_constant_parameters(du.ids[0])

{'param1': 1}

[TOP](#top)

<a id="DataUpload.delete"/>

### `DataUpload.delete()`

Delete file of given identifier.

In [114]:
DataUpload.delete?

Signature: DataUpload.delete(self, identifier)
Docstring:
REST/API DELETE request.
Delete item of the given identifier.
File:      /opt/conda/lib/python3.7/site-packages/viresclient/_api/upload.py
Type:      function


E.g. to delete the file previously uploaded:

In [115]:
du.delete(du.ids[0])

du.ids

[]

[TOP](#top)

<a id="DataUpload.clear"/>

### `DataUpload.clear()`

Delete all the uploaded files.

In [116]:
DataUpload.clear?

Signature: DataUpload.clear(self)
Docstring: Remove all uploaded items.  
File:      /opt/conda/lib/python3.7/site-packages/viresclient/_api/upload.py
Type:      function


In [117]:
du.clear()

du.ids

[]